# Import libraries

For openai>=1.0.0, openai.ChatCompletion is not supported anymore. Install an earlier version. Install cohere and tiktoken to address the Error message that recently came up:
<blockquote>ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.<br>
llmx 0.0.15a0 requires cohere, which is not installed.<br>
llmx 0.0.15a0 requires tiktoken, which is not installed.</blockquote>

In [ ]:
!pip install openai==0.27.8 cohere tiktoken
!pip install python-dotenv

In [ ]:
import inspect
import os
import dotenv
import openai
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directories to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/datasets/POPQUORN_gpt-3.5-turbo-0301',
            exist_ok=True)
os.makedirs('/content/drive/MyDrive/datasets/POPQUORN_gpt-3.5-turbo-0613',
            exist_ok=True)

# Authentication

You have to first save your OpenAI API key in a .env file in your Google Drive, in the format:
OPENAI_API_KEY = '...'.

In [ ]:
# OpenAI authentication on Google Colab. via
# https://stackoverflow.com/a/77166086
dotenv.load_dotenv('/content/drive/MyDrive/.env')
openai.api_key = os.getenv('OPENAI_API_KEY')

# Import dataset

In [ ]:
raw_data_texts = pd.read_csv(
    '/content/drive/MyDrive/datasets/raw_data_texts.csv')
raw_data_texts = raw_data_texts.reset_index(drop=True)

# Set prompts

In [ ]:
def system_content(persona):
    return inspect.cleandoc(
        f"""From now on, act as {persona}.
        Provide an answer that {persona} would regarding the comment.""")

In [ ]:
def user_content(comment):
    return inspect.cleandoc(f"""Comment: \"\"\"
    {comment}
    \"\"\"

    Considering the above comment on Reddit, how offensive do you think it is?

    Please tell us your answer using this scale from 1 (Not offensive at all)
    to 5 (Very offensive).

    Constraint: Even if you are uncertain, you must pick either “1”, “2”, “3”,
    “4”, or “5” without using any other words.""")

# Define a function to call Chat Completion API with (persona) or without (no-persona default) the *system message*

In [ ]:
# Prevent rate limit errors. via
# https://github.com/openai/openai-cookbook/blob/main/examples/
# How_to_handle_rate_limits.ipynb
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def inference_with_backoff(user_cont, model, system_cont=None):
    if system_cont is None:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_cont}
            ],
            temperature=0,
        )
    else:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": system_cont},
                {"role": "user", "content": user_cont}
            ],
            temperature=0,
        )
    return response['choices'][0]['message']['content']

# Define a function to perform model inference on **POPQUORN**

Create a function `predict_scores` that takes a list of personas (e.g., ['a man', 'a woman', 'a non-binary person']), a `topic` (e.g., a sociodemographic attribute such as *Gender*), and a GPT model name (e.g., gpt-3.5-turbo-0301) as required input arguments. The optional argument `base` determines if responses made by the no-persona default should be included. For each persona in the list of personas, the function iterates over each text in **POPQUORN**, calling the function `inference_with_backoff` for each text. It then adds the respective lists of scores predicted by each simulated persona as new columns to the **POPQUORN** dataframe. The dataframe is saved in TSV file format, and the function returns the dataframe.

In [ ]:
def predict_scores(persona_list, topic, model, base=None):
    df = raw_data_texts.copy(deep=True)
    comments = df['text'].values
    if base == 'Yes':
        df['predicted_scores_base'] = [
            inference_with_backoff(user_content(cmt),
                                   model) for cmt in comments]
    for persona in persona_list:
        persona += ' in the United States'
        df[f'predicted_scores_{persona}'.replace(' ', '_')] = [
            inference_with_backoff(user_content(cmt),
                                   model,
                                   system_content(persona)) for cmt in comments]
    df.to_csv(
        f'/content/drive/MyDrive/datasets/POPQUORN_{model}/'
        f'POPQUORN_predictions_{topic}_{model}.tsv',
        sep='\t', index=False)
    return df

# Create dictionaries, each with a `topic` as the key and a list of personas as the value

In [ ]:
gender = {
    'gender': ['a man',
               'a woman',
               'a non-binary person']}

In [ ]:
race_ethnicity = {
    'race_ethnicity': ['a White person',
                       'a Black or African American',
                       'an American Indian or Alaska Native',
                       'an Asian',
                       'a Native Hawaiian or Other Pacific Islander']}

In [ ]:
age_range = {
    'age_range': ['a person (aged <18)',
                  'a person (aged 18-24)',
                  'a person (aged 25-29)',
                  'a person (aged 30-34)',
                  'a person (aged 35-39)',
                  'a person (aged 40-44)',
                  'a person (aged 45-49)',
                  'a person (aged 50-54)',
                  'a person (aged 54-59)',
                  'a person (aged 60-64)',
                  'a person (aged >65)']}

In [ ]:
occupation = {
    'occupation': ['an employed person',
                   'an unemployed person',
                   'a student',
                   'a retiree',
                   'a homemaker',
                   'a self-employed person']}

In [ ]:
education_level = {
    'education_level': [
        'a person (whose education level is less than a high school diploma)',
        'a person (whose education level is a high school diploma or '
        'equivalent)',
        'a person (whose education level is a college degree)',
        'a person (whose education level is a graduate degree)']}

# Call the `predict_labels` function

Running each of the following cells could take several hours. Colab Pro+ includes continuous code execution (capped at 24 hours) and background execution capabilities, enabling you to close your browser/device while your code runs. CPU is sufficient. Alternatively, you can download the notebooks and edit the code accordingly to point to the directories on your local machine you want to use.

In [ ]:
predict_scores(
    list(gender.values())[0],
    list(gender.keys())[0],
    'gpt-3.5-turbo-0301',
    'Yes')

In [ ]:
predict_scores(
    list(gender.values())[0],
    list(gender.keys())[0],
    'gpt-3.5-turbo-0613',
    'Yes')

In [ ]:
predict_scores(
    list(race_ethnicity.values())[0],
    list(race_ethnicity.keys())[0],
    'gpt-3.5-turbo-0301')

In [ ]:
predict_scores(
    list(race_ethnicity.values())[0],
    list(race_ethnicity.keys())[0],
    'gpt-3.5-turbo-0613')

In [ ]:
predict_scores(
    list(age_range.values())[0],
    list(age_range.keys())[0],
    'gpt-3.5-turbo-0301')

In [ ]:
predict_scores(
    list(age_range.values())[0],
    list(age_range.keys())[0],
    'gpt-3.5-turbo-0613')

In [ ]:
predict_scores(
    list(occupation.values())[0],
    list(occupation.keys())[0],
    'gpt-3.5-turbo-0301')

In [ ]:
predict_scores(
    list(occupation.values())[0],
    list(occupation.keys())[0],
    'gpt-3.5-turbo-0613')

In [ ]:
predict_scores(
    list(education_level.values())[0],
    list(education_level.keys())[0],
    'gpt-3.5-turbo-0301')

In [ ]:
predict_scores(
    list(education_level.values())[0],
    list(education_level.keys())[0],
    'gpt-3.5-turbo-0613')